In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Work/jlProjects/MultivariantToolkit/forks/MultivariateInterpolation.jl`


┌ Warning: The active manifest file is missing a julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /Users/uwe/Work/jlProjects/MultivariantToolkit/forks/MultivariateInterpolation.jl/Manifest.toml:0


Precompiling project...


  ✓ MultivariateInterpolation
  1 dependency successfully precompiled in 1 seconds. 1 already precompiled.


In [2]:
using MultivariateInterpolation
using LinearAlgebra

In [3]:
DIM = 3
DEG = 2
LP = 2

mi = MultiIndexSet(DIM,DEG,LP)

3×11 MultiIndexSet{Matrix{Int64}, Int64}:
 0  1  2  0  1  0  0  1  0  1  0
 0  0  0  1  1  2  0  0  1  1  0
 0  0  0  0  0  0  1  1  1  1  2

In [4]:
exponents(m::MultiIndexSet) = MultiIndexSet.data

exponents (generic function with 1 method)

In [5]:
function polynomial_degree(multi_index_set,lp)
    @show multi_index_set
    norms = norm.(eachrow(multi_index_set), lp)
    maximum(norms)
end

polynomial_degree (generic function with 1 method)

In [6]:
polynomial_degree(mi,2)

multi_index_set = [0 1 2 0 1 0 0 1 0 1 0; 0 0 0 1 1 2 0 0 1 1 0; 0 0 0 0 0 0 1 1 1 1 2]


2.8284271247461903

In [7]:
sqrt(2^2+1)

2.23606797749979

In [8]:
MultivariateInterpolation._generate_exponents_static(3,2,2)

3×11 Matrix{Int64}:
 0  1  2  0  1  0  0  1  0  1  0
 0  0  0  1  1  2  0  0  1  1  0
 0  0  0  0  0  0  1  1  1  1  2

In [9]:
norm([0  2  1],2) <= 2

false

In [10]:
canidates_iter = MultivariateInterpolation._get_candidates(3, 2)

Base.Iterators.ProductIterator{Tuple{UnitRange{Int64}, UnitRange{Int64}, UnitRange{Int64}}}((0:2, 0:2, 0:2))

In [11]:
for el in canidates_iter
    println(el)
    println(norm(el,2)<=2)
    println()
end

(0, 0, 0)
true

(1, 0, 0)
true

(2, 0, 0)
true

(0, 1, 0)
true

(1, 1, 0)
true

(2, 1, 0)
false

(0, 2, 0)
true

(1, 2, 0)
false

(2, 2, 0)
false

(0, 0, 1)
true

(1, 0, 1)
true

(2, 0, 1)
false

(0, 1, 1)
true

(1, 1, 1)
true

(2, 1, 1)
false

(0, 2, 1)
false

(1, 2, 1)
false

(2, 2, 1)
false

(0, 0, 2)
true

(1, 0, 2)
false

(2, 0, 2)
false

(0, 1, 2)
false

(1, 1, 2)
false

(2, 1, 2)
false

(0, 2, 2)
false

(1, 2, 2)
false

(2, 2, 2)
false



In [12]:
result_iter = Iterators.filter(n -> norm(n, 2) <= 2, canidates_iter)

Base.Iterators.Filter{var"#9#10", Base.Iterators.ProductIterator{Tuple{UnitRange{Int64}, UnitRange{Int64}, UnitRange{Int64}}}}(var"#9#10"(), Base.Iterators.ProductIterator{Tuple{UnitRange{Int64}, UnitRange{Int64}, UnitRange{Int64}}}((0:2, 0:2, 0:2)))

In [13]:
for el in result_iter
    println(el)
end

(0, 0, 0)
(1, 0, 0)
(2, 0, 0)
(0, 1, 0)
(1, 1, 0)
(0, 2, 0)
(0, 0, 1)
(1, 0, 1)
(0, 1, 1)
(1, 1, 1)
(0, 0, 2)


In [14]:
MultivariateInterpolation._flat(result_iter)

3×11 Matrix{Int64}:
 0  1  2  0  1  0  0  1  0  1  0
 0  0  0  1  1  2  0  0  1  1  0
 0  0  0  0  0  0  1  1  1  1  2

In [15]:
for col in eachcol(mi)
    println(col)
end

[0, 0, 0]
[1, 0, 0]
[2, 0, 0]
[0, 1, 0]
[1, 1, 0]
[0, 2, 0]
[0, 0, 1]
[1, 0, 1]
[0, 1, 1]
[1, 1, 1]
[0, 0, 2]


In [16]:
typeof(Inf) <: Real

true

In [43]:
mi = MultiIndexSet(3,3,2)

3×29 MultiIndexSet{Matrix{Int64}, Int64}:
 0  1  2  3  0  1  2  0  1  2  0  0  1  …  0  1  2  0  1  2  0  1  2  0  1  0
 0  0  0  0  1  1  1  2  2  2  3  0  0     2  2  2  0  0  0  1  1  1  2  2  0
 0  0  0  0  0  0  0  0  0  0  0  1  1     1  1  1  2  2  2  2  2  2  2  2  3

In [44]:
mi.lp

2

In [55]:
function contains(mi::MultiIndexSet,x::AbstractVector{T}) where T<:Real
    if length(x)!=spatial_dimension(mi)
        return false
    end
    for col in eachcol(mi)
        if isapprox(col,x)
            
            return true
        end
    end
    return false
end

function contains(mi::MultiIndexSet,x::AbstractMatrix{T}) where T<:Real
    res = zeros(Bool,size(x,2))
    for i in 1:size(x,2)
        if contains(mi,view(x,:,i))
            res[i] = true
        end
    end
    return res
end

contains (generic function with 3 methods)

In [61]:
all(contains(mi,MultiIndexSet(3,3,3)))

false

In [71]:
mi = MultiIndexSet(    3, 1 , 0.5)

1×2 MultiIndexSet{Matrix{Int64}, Float64}:
 0  1

In [76]:
mi2 = MultiIndexSet(  1, 1,0.5)

1×2 MultiIndexSet{Matrix{Int64}, Float64}:
 0  1

In [68]:
contains(mi2,mi)

15-element Vector{Bool}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [70]:
Inf<=2

false